# Interatomic Potential Repository Content Manager

This is a working Notebook providing a single location for adding and modifying content in the potentials.nist.gov database.

Notebook outline:

0. Setup: Python library imports and defining global parameters.

1. Citations: Load, add and modify citation information associated with potential models. Each potential should have at least one citation, with the first being the "primary" one for generating IDs. For unpublished potentials, use the unpublished citation option.

2. Potentials: Load, add and modify potential information.  A "potential" should be a unique parameterization with an associated citation. Note that saving to the database is done in step #4.

3. Implementations: Load, add and modify the implementations of a potential. An "implementation" is a specific version/representation of a potential in a specific format. Each potential can have zero, one or multiple implementations.

4. Save potentials: Review the new/modified potentials and save the content to the database.

5. Actions: Actions are used to list changes to the repository. If a potential is added/modified, a corresponding action should also be added.

6. Requests: A request is listed on the repository whenever someone asks for a model that we do not have.

## 0. Setup

### Library imports

In [1]:
import uuid
import datetime
import shutil
from pathlib import Path
import socket

import potentials

import numpy as np
import pandas as pd

# Jupyter display libraries
from IPython.display import display, HTML

### Global parameters

- __complete_potentials__ is a compiled list of the potentials whose addition/changes were uploaded to the database.  This is used by step #5 for reporting website changes.  This should be reset after creating an Action for meaningful sets of potentials.

- __db__ is the Database object for accessing and uploading records to the database.  remote_name accesses the saved "potentials" settings, which has sign-in info for https://potentials.nist.gov.

- __workspace__ is the workspace to assign to the uploaded records.  If None, then they are only accessible to the submitter until the workspace is changed.  If 'Global Public Workspace', anyone can see the records after uploading.

In [2]:
complete_potentials = []

In [3]:
db = potentials.Database(local=True, remote=True, remote_name='potentials')

In [4]:
workspace = 'Global Public Workspace'

- - -

## 1. Citations

### 1.1 Build or load citation

#### Option #1: For existing citations and/or new dois

In [5]:
doi = '10.1016/j.commatsci.2024.113595'

citation = db.fetch_citation(doi, verbose=True)

Matching record retrieved from local


#### Option #2: For new doi-less citations

In [33]:
ID = note = f'2024--Starikov-S-Grigorev-P-Olsson-P'

citation = potentials.load_record('Citation',
    ENTRYTYPE = 'article',
    title = 'Angular-dependent interatomic potential for large-scale atomistic simulation of W-Mo-Nb ternary alloys',
    author = 'S. Starikov and P. Grigorev and P. Olsson',
    ID = ID,
    note = note,
    year = 2024,
    journal = 'Computational Materials Science',
    #volume = 0,
    pages = 'accepted',
    doi = 'tbd',
    abstract=' '.join(['We present a new classical interatomic potential designed for simulation of',
                       'the W-Mo-Nb system. The angular-dependent format of the potential allows',
                       'for reproduction of many important properties of pure metals and complex',
                       'concentrated alloys with good accuracy. Special attention during the development',
                       'and validation of the potential was paid to the description of vacancies,',
                       'screw dislocations and planar defects, as well as thermo-mechanical',
                       'properties. Here, the applicability of the developed model is demonstrated',
                       'by studying the temperature dependence of the elastic moduli and average',
                       'atomic displacement in pure metals and concentrated alloys up to the melting point.'])
)
citation.build_model()
print(citation.model.json(indent=4))

{
    "citation": {
        "document-type": "journal",
        "title": "Angular-dependent interatomic potential for large-scale atomistic simulation of W-Mo-Nb ternary alloys",
        "author": [
            {
                "given-name": "S.",
                "surname": "Starikov"
            },
            {
                "given-name": "P.",
                "surname": "Grigorev"
            },
            {
                "given-name": "P.",
                "surname": "Olsson"
            }
        ],
        "publication-name": "Computational Materials Science",
        "publication-date": {
            "year": 2024
        },
        "abstract": "We present a new classical interatomic potential designed for simulation of the W-Mo-Nb system. The angular-dependent format of the potential allows for reproduction of many important properties of pure metals and complex concentrated alloys with good accuracy. Special attention during the development and validation of the potential

In [32]:
author = 'Gabriel Plummer and Jacob P. Tavenner and Mikhail I. Mendelev and Zhigang Wu and John W. Lawson'
year = 2024
ID = note = f'{year}--Mendelev-M-I-Plummer-G--Ni-O'
title = 'to be published'

citation = potentials.load_record('Citation',
    ENTRYTYPE = 'unpublished',
    title = title,
    author = author,
    ID = ID,
    note = note,
    year = year,
)
citation.build_model()

DataModelDict([('citation',
                DataModelDict([('document-type', 'unspecified'),
                               ('title', 'to be published'),
                               ('author',
                                [DataModelDict([('given-name', 'G.'),
                                                ('surname', 'Plummer')]),
                                 DataModelDict([('given-name', 'J.P.'),
                                                ('surname', 'Tavenner')]),
                                 DataModelDict([('given-name', 'M.I.'),
                                                ('surname', 'Mendelev')]),
                                 DataModelDict([('given-name', 'Z.'),
                                                ('surname', 'Wu')]),
                                 DataModelDict([('given-name', 'J.W.'),
                                                ('surname', 'Lawson')])]),
                               ('publication-date',
                            

#### Option #3: From local directory

In [ ]:
libdir = Path('E:/website/LAMMPS potentials/2nn/biblib')

citations = []
for bibfile in libdir.glob('*.bib'):
    with open(bibfile, encoding='UTF-8') as f:
        citations.append(potentials.load_record('Citation', model=f.read()))

#### Option #4: From currently loaded potentials

In [ ]:
#citation = complete_potentials[0].citations[0]
#print(citation.year_authors)

### 1.2 Investigate and modify

- Add abstract

- Check and replace garbled latex or other symbols in authors, title and abstract

- Verify correct year and page number(s)

#### 1.2.1 Generate html and list content fields

In [6]:
print(citation.year_authors)
print()
citation.html(render=True)
print()
for key, value in citation.bib.items():
    print(key+':', value)

2025--Sharifi-H-Wick-C-D




year: 2025
volume: 248
url: http://dx.doi.org/10.1016/j.commatsci.2024.113595
title: Developing interatomic potentials for complex concentrated alloys of Cu, Ti, Ni, Cr, Co, Al, Fe, and Mn
publisher: Elsevier BV
pages: 113595
month: February
journal: Computational Materials Science
issn: 0927-0256
doi: 10.1016/j.commatsci.2024.113595
author: Sharifi, Hamid and Wick, Collin D.
abstract: Complex concentrated alloys (CCAs) are a new generation of metallic alloys composed of three or more principal elements with physical and mechanical properties that can be tuned by adjusting their compositions. The extensive compositional workspace of CCAs makes it impractical to perform a comprehensive search for a specific material property using experimental measurements. The use of computational methods can rapidly narrow down the search span, improving the efficiency of the design process. We carried out a high-throughput parameterization of modified embedded atom method (MEAM) interatomic potentia

#### 1.2.2 Modify and update fields

Any valid bibtex fields can be added, but citation rendering relies only on the primary fields. 

In [7]:
#citation.bib['author'] = "Fan, Zhengxuan and Émile Maras and Cottura, Maeva and Marinica, Mihai-Cosmin and Clouet, Emmanuel"
#citation.bib['title'] = 'Grain Size Effects on the Deformation of α-Fe Nanopolycrystals: Massively Large-Scale Molecular Dynamics Simulations Using Machine Learning Interatomic Potential'
citation.bib['abstract'] = ' '.join([
    "Complex concentrated alloys (CCAs) are a new generation of metallic alloys composed of three or more principal",
    "elements with physical and mechanical properties that can be tuned by adjusting their compositions. The",
    "extensive compositional workspace of CCAs makes it impractical to perform a comprehensive search for a",
    "specific material property using experimental measurements. The use of computational methods can rapidly",
    "narrow down the search span, improving the efficiency of the design process. We carried out a high-throughput",
    "parameterization of modified embedded atom method (MEAM) interatomic potentials for combinations of Cu, Ti,",
    "Ni, Cr, Co, Al, Fe, and Mn using a genetic algorithm. Unary systems were parameterized based on DFT calculations",
    "and experimental results. MEAM potentials for 28 binary and 56 ternary combinations of the elements",
    "were parameterized to DFT results that were carried out with semi-automated frameworks. Specific attention was",
    "made to reproduce properties that impact compositional segregation, material strength, and mechanics."])
#citation.bib['pages'] = '255'
#citation.bib['issue'] = '1'
#citation.bib['year'] = '2021'
#citation.bib['volume']
#citation.bib['number']
#citation.bib['doi'] = '10.1016/j.jssc.2022.123096'
#citation.bib['ENTRYTYPE'] = 'article'
#citation.bib['journal'] = 'arXiv'

#### 1.2.3 Review and Save Citation

In [8]:
citation.build_model()
print(citation.year_authors)
citation.html(render=True)

2025--Sharifi-H-Wick-C-D


In [9]:
db.save_citation(citation, overwrite=True, verbose=True)
db.upload_citation(citation, workspace=workspace, overwrite=True, verbose=True)

Citation record named 10.1016_j.commatsci.2024.113595 added to C:\Users\lmh1\Documents\library
Citation record named 10.1016_j.commatsci.2024.113595 added to https://potentials.nist.gov/
Citation record named 10.1016_j.commatsci.2024.113595 assigned to workspace Global Public Workspace


In [30]:
#db.delete_citation(citation, local=True, remote=True, verbose=True)

Citation record named 2024--mendelev-m-i-plummer-g--ni-o deleted from C:\Users\lmh1\Documents\library
Citation record named 2024--mendelev-m-i-plummer-g--ni-o deleted from https://potentials.nist.gov/


In [ ]:
#db.download_citations(overwrite=True, verbose=True)

In [7]:
citations = []

In [8]:
citations.append(citation)

- - -

## 2. Potentials

### 2.1 Build or load potential

#### Option #1: Build new potential using citation(s)

- __elements__ (*list*) The element models used by the potential.
- __citations__ (*list*) The citation(s) to associate with the potential.
- __notes__ (*str, optional*) Notes providing more information on the potential, such as usage notes and how the potential relates to other potentials.
- __key__ (*str, optional*) UUID4 key to assign to the potential.  If None, a new key will be generated.
- __othername__ (*str, optional*) Specifies an alternate description of the interactions besides the list of elements.  Examples include if the model is "universal", designed for a specific compound, or is a coarse-grain representation.
- __fictional__ (*bool, optional*) Flag indicating if the model is "fictional", i.e. purposefully fit to unrealistic properties. 
- __modelname__ (*str, optional*) Used if citation + element info is not enough to create a unique ID, e.g. a publication lists multiple parameterizations of the same element. The value should make it clear which version it is associated with, and correspond to how the authors refer to it when possible.
- __recorddate__ (*datetime.date, optional*) Date to assign to the record.  If not given, will use today's date.

In [10]:
potential = potentials.load_record('Potential',
    elements=['Fe',],# 'Mn', 'Ni', 'Ti', 'Cu', 'Cr', 'Co', 'Al'],
    citations=citations,
    notes=' '.join([
        "This is the elemental listing for the 2025--Sharifi-H-Wick-C-D--Fe-Mn-Ni-Ti-Cu-Cr-Co-Al potential.  This potential focuses on mechanical properties.",
        "However, the potential was not optimized for temperature-dependent properties and was not fit to density, thermal expansion coefficients, or thermal conductivity data."
    ]),
    #key=None,
    #othername='Ge60Se40',
    #fictional=True,
    #modelname='22',
    #recorddate=datetime.date(2019, 12, 24)
    )
potential.url = f'https://potentials.nist.gov/pid/rest/local/potentials/{potential.name}'
potential.build_model()
None

#### Option #2: Select existing potential

In [13]:
potential = db.get_potential(id='2024--Plummer-G-Tavenner-J-P-Mendelev-M-I-et-al--Ni-O', verbose=True)

Matching record retrieved from local


In [31]:
#db.delete_potential(potential, local=True, remote=True, verbose=True)

Potential record named potential.2024--Mendelev-M-I-Plummer-G--Ni-O deleted from C:\Users\lmh1\Documents\library
Potential record named potential.2024--Mendelev-M-I-Plummer-G--Ni-O deleted from https://potentials.nist.gov/


#### Option #3: Get currently loaded potential

In [ ]:
#potential = complete_potentials[1]
#print(potential.name)

### 2.2 Investigate and modify

#### 2.2.1 Generate html and list content fields

In [11]:
potential.html(render=True)

In [13]:
#potential.citations.append(citation)
potential.build_model()
None

DataModelDict([('interatomic-potential',
                DataModelDict([('key', '0c44bfc0-1e71-4199-98fc-568ca5f51957'),
                               ('id',
                                '2024--Ito-K-Yokoi-T-Hyodo-K-Mori-H--Fe-22'),
                               ('URL',
                                'https://potentials.nist.gov/pid/rest/local/potentials/potential.2024--Ito-K-Yokoi-T-Hyodo-K-Mori-H--Fe-22'),
                               ('record-version', '2024-11-15'),
                               ('description',
                                DataModelDict([('citation',
                                                [DataModelDict([('document-type',
                                                                 'journal'),
                                                                ('title',
                                                                 'Machine learning interatomic potential with DFT accuracy for general grain boundaries in α-Fe'),
              

#### 2.2.2 Modify and update fields

Same fields as above

In [17]:
#potential.key = 
#potential.recorddate = datetime.date()
#potential.elements = []
#potential.othername = 'water'
#potential.fictional = 
#potential.modelname = 
potential.notes = ' '.join([
    "This potential is designed for mechanical analysis, including shear strength and elastic constants, dislocation dynamics and their impact on alloy strength, and the analysis of defect effects, such as voids, on material properties.",
    "However, the potential was not optimized for temperature-dependent properties and was not fit to density, thermal expansion coefficients, or thermal conductivity data."])
#potential.notes += ' The associated publication has a supplementary information document which includes thorough testing of library appropriateness of liquid properties, and comments and analyses on numerical stability and convergence'
#potential.notes += '"'
#potential.citations[0] = citation
#potential.name = f'potential.{potential.id}'
potential.build_model()
None

## 3. Implementations

### 3.1 List all current implementation types

List is given to help keep type values (relatively) uniform. Only needs to be called after new types have been added.

In [30]:
pots = db.get_potentials(remote=False)
imptypes = []
for pot in pots:
    for imp in pot.implementations:
        imptypes.append(imp.type)
imptypes = np.unique(imptypes)

print('All existing implementation types:')
for imptypes in imptypes:
    print('   ', imptypes)

All existing implementation types:
    ADP tabulated functions
    ASE calculator
    Dynamo MEAM
    EAM setfl
    EAM tabulated functions
    Equations
    FORTRAN
    Finnis-Sinclair tables
    GAP
    GULP
    IMD option EAM
    LAMMPS pair_style adp
    LAMMPS pair_style aenet (custom)
    LAMMPS pair_style agni
    LAMMPS pair_style bop
    LAMMPS pair_style comb3
    LAMMPS pair_style eam
    LAMMPS pair_style eam/alloy
    LAMMPS pair_style eam/cd
    LAMMPS pair_style eam/fs
    LAMMPS pair_style eam/he
    LAMMPS pair_style edip
    LAMMPS pair_style edip/multi
    LAMMPS pair_style eim
    LAMMPS pair_style extep
    LAMMPS pair_style hdnnp
    LAMMPS pair_style hybrid table linear 1000 eam/alloy
    LAMMPS pair_style hybrid/overlay eam/alloy eam/fs
    LAMMPS pair_style hybrid/overlay zbl eam/alloy
    LAMMPS pair_style hybrid/overlay zbl snap
    LAMMPS pair_style lcbop
    LAMMPS pair_style meam
    LAMMPS pair_style meam (modified)
    LAMMPS pair_style meam/spline
    L

### 3.2 Build or load implementation

#### 3.2.1 List ids for current potential and implementations

In [12]:
print('suggested imp prefix:', potential.impid_prefix)
print('implementation ids:')
for i, implementation in enumerate(potential.implementations):
    print(f'{i}: {implementation.id} {implementation.type}')

suggested imp prefix: 2025--Sharifi-H--Fe
implementation ids:


In [86]:
#del potential.implementations[0]

#### 3.2.2 Build new implementation

- __type__ (*str*) Describes the type of the implementation. Use one of the listed values above if possible.
- __id__ (*str*) Unique human-readable id for the implementation. For new content, should be derived from the associated potential id:
    - Remove all authors except for the first
    - Add a simple format descriptor: LAMMPS, GULP, table, parameters, FORTRAN, etc.
    - Add a version descriptor: ipr1, ipr2, etc
- __notes__ (*str, optional*) Notes on the implementation.  This includes where the file(s) came from, who created them, testing info, how it differs from other versions, etc.
- __key__ (*str, optional*) UUID4 key to assign to the implementation.  If None, a new key will be generated.
- __status__ (*str, optional*) Indicates the status of the implementation. Available values are 
    - "active" (default) indicates a current implementation.
    - "superseded" indicates an implementation that is still consistent with the potential model, but has minor issues that were fixed by a newer implementation.
    - "retracted" indicates an implementation that was identified as being an invalid representation of the potential model.
- __date__ (*datetime.date, optional*) The date that the implementation was submitted or added.  If not given, will use today's date.

In [13]:
potential.add_implementation(
    type = 'LAMMPS pair_style meam',
    id = f'{potential.impid_prefix}--LAMMPS--ipr1',
    notes = ' '.join([
        "These files were provided by Hamid Sharifi on February 11, 2025."
    ]),
    #key='0cdc699f-e78a-49db-8606-7a33c98c2184',
    #status=None,
    #date=datetime.date(2020, 6, 18), 
)
index = len(potential.implementations) - 1

In [27]:
potential.add_implementation(
    type = 'ASE calculator',
    id = f'{potential.impid_prefix}--equations--ipr1',
    notes = ' '.join([
        'This file and equations were provided by Alexey Verkhovtsev on March 19, 2024.  The .pdf collects the equation and parameter values as described in the paper.  The .pot file are the parameters as used by the MBN Explorer software package.'
    ]),
    #key='0cdc699f-e78a-49db-8606-7a33c98c2184',
    #status=None,
    #date=datetime.date(2020, 6, 18), 
)
index = len(potential.implementations) - 1

In [28]:
kimid = 'MO_959249795837'
implementation = potentials.Implementation(
    type='OpenKIM',
    id=kimid,
    notes=' '.join(['Listing found at https://openkim.org.']),
    #notes=' '.join(['Listing found at https://openkim.org.  This KIM potential is based on a parameter file with identical parameter values as 1989--Tersoff-J--Si-Ge--LAMMPS--ipr1.']),
    #notes=' '.join(['Listing found at https://openkim.org. This KIM potential is implemented from the analytical expressions rather than a tabulated parameter file.'])#  The parameter file that this KIM potential is based on has slightly different values due to precision rounding than 2017--Purja-Pun-G-P--Si--LAMMPS--ipr1.']),
    key='da76eb36-7ca1-421a-bd64-8f6bdd905b15',
    #status=None,
    #date=datetime.date(2020, 6, 18), 
)
implementation.add_link(url=f"https://openkim.org/id/{kimid}", linktext = kimid)

index = len(potential.implementations)
potential.implementations.append(implementation)



In [14]:
potential.build_model()
potential.html(render=True)

#### 3.2.3 Select implementation

Must be done

In [15]:
index = 0
implementation = potential.implementations[index]

### 3.3 Modify implementation

#### 3.3.1 Modify implementation metadata 

In [16]:
implementation.notes

'These files were provided by Mikhail Mendelev on October 22, 2024. Note that as of this post, coul/ctip is in the process of being added to LAMMPS and therefore is not yet available in the most recent stable release. If you would like to use this potential, contact Gabriel Plummer (gabriel.w.plummer@nasa.gov) for details on how to use it with LAMMPS.'

In [17]:
implementation.notes = 'These files were provided by Mikhail Mendelev on October 22, 2024. Update: coul/ctip is available in LAMMPS starting with the 19Nov2024 release.'

In [22]:
implementation.type = 'deepmd-kit'
#implementation.key=None
#implementation.date=datetime.date(2020, 1, 10)
#implementation.id = f'{potential.impid_prefix}--LAMMPS--ipr1'
#implementation.notes = ' '.join(['This file was provided by Giovanni Bonny on April 3, 2024 with the agreement of all the authors.'])
#implementation.notes += " Update July 19 2023: This version has been superseded as it had the incorrect atomic mass for H."

#implementation.status='superseded'

#### 3.3.2 Add implementation content

The content for the implementation can exist in one of the following formats

1. Artifacts: files that are hosted on the potentials repository.
2. Parameters: list of the potential's parameters to show as html text on the repository.
3. WebLinks: hyperlinks to content hosted externally.

#### Option #1: Add artifacts

File path parameters

- __newpotpath__ is the path to the directory where the new potential files are located.
- __localpath__ is the path to the local copy of the NIST potentials repository.
- __webpath__ is the url path for the NIST potentials repository that coincides with the localpath on the machine.
- __relpath__ is the relative path from localpath where the files are copied to, and the relative path from webpath where they will be found once uploaded to the website.

In [16]:
# Paths to files on desktop
machine = socket.gethostname()
if machine == 'PG903193':
    newpotpath = Path('F:/website/new potentials')
    localpath = Path('F:/website/IPR-website/potentials/')

# Paths to files on laptop
elif machine == 'PG902164':
    newpotpath = Path('C:/Users/lmh1/Documents/website/new potentials')
    localpath = Path('C:/Users/lmh1/Documents/website/IPR-website/potentials/')

# Web and relative paths
webpath = 'https://www.ctcms.nist.gov/potentials/'
relpath = f'Download/{potential.id}/{index+1}/'

Each artifact has:

- __filename__ (*str*) The name of the file (no path information).
- __url__ (*str*) The absolute url where the file can be downloaded from.
- __label__ (*str, optional*) An optional descriptive label for the file.

In [17]:
for potfile in newpotpath.glob('*'):
    if potfile.is_file():
        print(potfile.name)

1-s2.0-S0927025624008164-mmc1.docx
Al.meam
Co.meam
Cr.meam
Cu.meam
Fe.meam
library.meam
Mn.meam
Ni.meam
Ti.meam


In [18]:
# Specify files from newpotpath to add
filenames = [
    Path(newpotpath, 'library.meam'),
    Path(newpotpath, 'Fe.meam'),
    #Path(newpotpath, 'Al_pot.snapparam'),
    ]

labels = [None for i in range(len(filenames))]

In [146]:
# Specify files from newpotpath to add
filenames = [
    Path(newpotpath, 'READ_ME.txt'),
    Path(newpotpath, 'FeCCr_d.eam.alloy'),
    Path(newpotpath, 'FeCCr_s.eam.fs'),
]

labels = [
    'Documentation',
    'd_band',
    's_band'
]

In [98]:
# Specify files from newpotpath to add
filenames = [
    Path(newpotpath, 'F_Fe.spt'),
    Path(newpotpath, 'F_Cu.spt'),
    Path(newpotpath, 'F_Ni.spt'),
    Path(newpotpath, 'F_Mn.spt'),
    Path(newpotpath, 'rhoFe.spt'),
    Path(newpotpath, 'rhoCu.spt'),
    Path(newpotpath, 'rhoNi.spt'),
    Path(newpotpath, 'rhoMn.spt'),
    Path(newpotpath, 'pFeFe.spt'),
    Path(newpotpath, 'pCuCu.spt'),
    Path(newpotpath, 'pNiNi.spt'),
    Path(newpotpath, 'pMnMn.spt'),
    Path(newpotpath, 'pFeCu.spt'),
    Path(newpotpath, 'pFeNi.spt'),
    Path(newpotpath, 'pFeMn.spt'),
    Path(newpotpath, 'pCuNi.spt'),
    Path(newpotpath, 'pCuMn.spt'),
    Path(newpotpath, 'pNiMn.spt'),
    ]

labels = [
    'Fe F(ρ)',
    'Cu F(ρ)',
    'Ni F(ρ)',
    'Mn F(ρ)',
    'Fe ρ(r)',
    'Cu ρ(r)',
    'Ni ρ(r)',
    'Mn ρ(r)',
    'Fe φ(r)',
    'Cu φ(r)',
    'Ni φ(r)',
    'Mn φ(r)',
    'Fe-Cu φ(r)',
    'Fe-Ni φ(r)',
    'Fe-Mn φ(r)',
    'Cu-Ni φ(r)',
    'Cu-Mn φ(r)',
    'Ni-Mn φ(r)'
]

In [19]:
# Make download directory in local copy of website
downloadpath = Path(localpath, relpath)
if not downloadpath.is_dir():
    downloadpath.mkdir(parents=True)

# Copy files and add artifact listings
for filename, label in zip(filenames, labels):
    # Build url
    url = webpath + relpath + filename.name
    # Copy files
    shutil.copy(filename, downloadpath)
    print(Path(downloadpath, filename.name))
    
    # Add artifact listing to implementation
    implementation.add_artifact(filename=filename.name, url=url, label=label)

F:\website\IPR-website\potentials\Download\2025--Sharifi-H-Wick-C-D--Fe\1\library.meam
F:\website\IPR-website\potentials\Download\2025--Sharifi-H-Wick-C-D--Fe\1\Fe.meam


#### Option #1b: Update file paths

In [43]:
#newdir= Path(localpath, relpath)
#if not newdir.is_dir():
#    newdir.mkdir(parents=True)

#for artifact in implementation.artifacts:
    # Copy artifact file to new location
#    oldpath = Path(localpath, artifact.url.replace(webpath, ''))
    
#    newpath = Path(newdir, artifact.filename)
    #shutil.copy(oldpath, newpath)
    
    # Update the url
#    artifact.url = webpath + relpath + artifact.filename

#### Option #2: Add parameters

Each parameter has:

- __name__ (*str, optional*) The name of the parameter or string parameter line.
- __value__ (*float, optional*) The value of the parameter.
- __unit__ (*str, optional*) Units associated with value.

Note: if the parameters are purely text-based, you can use only the name field.

#### Option #3: Add weblinks

Each weblink has:

- __url__ (*str*) URL for the link.
- __label__ (*str, optional*) A short description label to proceed the link.
- __linktext__ (*str, optional*) The text for the link, i.e. what gets clicked on.

In [61]:
#url = 'https://github.com/mengfsou/MLPs-FeC'
#label = 'github repository'
#linktext = 'https://github.com/mengfsou/MLPs-FeC'

#implementation.add_link(url=url, label=label, linktext=linktext)

#url = 'https://docs.deepmodeling.com/projects/deepmd/en/master/index.html'
#label = 'DeePMD-kit documentation'
#linktext = 'https://docs.deepmodeling.com/projects/deepmd/en/master/index.html'

#implementation.add_link(url=url, label=label, linktext=linktext)


#url = 'https://doi.org/10.5281/zenodo.10131449'
#label = 'Zenodo training data'
#linktext = 'https://doi.org/10.5281/zenodo.10131449'

#implementation.add_link(url=url, label=label, linktext=linktext)


url = 'https://gitlab.com/ashapeev/interface-lammps-mlip-2'
label = 'LAMMPS-MLIP interface gitlab'
linktext = 'https://gitlab.com/ashapeev/interface-lammps-mlip-2'

implementation.add_link(url=url, label=label, linktext=linktext)

In [ ]:
#implementation.links.pop()

In [82]:
#implementation.links[0].linktext=linktext

## 4. Review and save potentials

In [20]:
potential.build_model()
potential.html(render=True)

In [21]:
db.save_potential(potential, overwrite=True, verbose=True)
with db.remote_database.cdcs.auto_set_pid_off():
    db.upload_potential(potential, workspace=workspace, overwrite=True, verbose=True)
complete_potentials.append(potential)

Potential record named potential.2025--Sharifi-H-Wick-C-D--Fe added to C:\Users\lmh1\Documents\library
Potential record named potential.2025--Sharifi-H-Wick-C-D--Fe added to https://potentials.nist.gov/
Potential record named potential.2025--Sharifi-H-Wick-C-D--Fe assigned to workspace Global Public Workspace


In [64]:
[pot.id for pot in complete_potentials]

['2024--Ito-K-Yokoi-T-Hyodo-K-Mori-H--Fe-16',
 '2024--Ito-K-Yokoi-T-Hyodo-K-Mori-H--Fe-18',
 '2024--Ito-K-Yokoi-T-Hyodo-K-Mori-H--Fe-20',
 '2024--Ito-K-Yokoi-T-Hyodo-K-Mori-H--Fe-22']

In [ ]:
#db.delete_potential(potential, local=True, remote=True, verbose=True)
#complete_potentials.pop(-1)

In [ ]:
#db.download_potentials(overwrite=True, format='json', indent=4, verbose=True)

In [23]:
#complete_potentials.append(potential)

## 5. potential_LAMMPS

This section generates the potential_LAMMPS metadata records for generating the LAMMPS input command lines for supported potentials.

### 5.1 Build potential_LAMMPS

#### Specify parameters

Common settings

- __id__ (*str, optional*) implementation id. Should match with the implementation id from above.
- __key__ (*str, optional*) implementation key. Should match with the implementation key from above.
- __potid__ (*str, optional*) potential id. Should match with the potential id from above.
- __potkey__ (*str, optional*) potential key. Should match with the potential key from above.
- __elements__ (*list, optional*) The elements that the implementation simulates.  Should match with the elements from above if the implementation explicitly models each element.
- __symbols__ (*list, optional*) The particle model symbols defined by the implementation.  Not needed if identical to elements.
- __masses__ (*list, optional*) The masses to assign to each symbol/element.  Should be given if the implementation specifies the mass.  Required if elements is not given.
- __units__ (*str, optional*) The LAMMPS units to use with the implementation.  Default value is 'metal'.
- __atom_style__ (*str, optional*) The LAMMPS atom_style to use with the implementation.  Default value is 'atomic'.
- __pair_style__ (*str*) The LAMMPS pair_style associated with the implementation.
- __pair_style_terms__ (*list, optional*) Any additional terms that should appear on the pair_style command line.
- __comments__ (*str, optional*) Info about where the potential is hosted, to be printed when LAMMPS commands are generated.
- __dois__ (*list, optional*) A list of publication DOIs associated with the potential. 
- __artifacts__ (*list, optional*) A list of Artifact objects for the associated potential files.

In [22]:
kwargs = {}
kwargs['id'] = implementation.id
kwargs['key'] = implementation.key
kwargs['potid'] = potential.id
kwargs['potkey'] = potential.key

kwargs['elements'] = kwargs['symbols'] = potential.elements
#kwargs['elements'] = ['Si', 'Fe']
#kwargs['symbols'] = ['Fe', 'nmFe']
kwargs['masses'] = [55.845]#, 54.94, 58.6934, 47.880, 63.546, 51.960, 58.9330, 26.9815]

#kwargs['units'] =
#kwargs['atom_style'] = 

kwargs['pair_style'] = implementation.type.replace('LAMMPS pair_style ', '')
#kwargs['pair_style_terms'] = ['spline', 10000]

kwargs['comments'] = '\n'.join([f'Potential {implementation.id} listed in the NIST Interatomic Potentials Repository:',
                                f'https://www.ctcms.nist.gov/potentials/entry/{potential.id}/{implementation.id}.html',
                                f''])

#kwargs['dois'] = []
#for cit in potential.citations:
#    kwargs['dois'].append(cit.bib['doi'])

kwargs['artifacts'] = implementation.artifacts

Option #1 For pair potentials without parameter files, e.g lj/cut

- __interactions__ (*list*) Lists parameters associated with each unique interaction.  Each value is a dict with
    - __symbols__ (*list*) The two elemental model symbols to associate with the interaction.
    - __terms__ (*list*) The terms that appear on the pair_coeff command line for that interaction.

In [34]:
kwargs['interactions'] = [
    {'symbols': ['Co', 'Co'], 'terms': ['mie_Co-Co.table', 'ENTRY']},
    {'symbols': ['Cr', 'Cr'], 'terms': ['mie_Cr-Cr.table', 'ENTRY']},
    {'symbols': ['Fe', 'Fe'], 'terms': ['mie_Fe-Fe.table', 'ENTRY']},
    {'symbols': ['Mn', 'Mn'], 'terms': ['mie_Mn-Mn.table', 'ENTRY']},
    {'symbols': ['Ni', 'Ni'], 'terms': ['mie_Ni-Ni.table', 'ENTRY']},
    {'symbols': ['Co', 'Cr'], 'terms': ['mie_Co-Cr.table', 'ENTRY']},
    {'symbols': ['Co', 'Fe'], 'terms': ['mie_Co-Fe.table', 'ENTRY']},
    {'symbols': ['Co', 'Mn'], 'terms': ['mie_Co-Mn.table', 'ENTRY']},
    {'symbols': ['Co', 'Ni'], 'terms': ['mie_Co-Ni.table', 'ENTRY']},
    {'symbols': ['Cr', 'Fe'], 'terms': ['mie_Cr-Fe.table', 'ENTRY']},
    {'symbols': ['Cr', 'Mn'], 'terms': ['mie_Cr-Mn.table', 'ENTRY']},
    {'symbols': ['Cr', 'Ni'], 'terms': ['mie_Cr-Ni.table', 'ENTRY']},
    {'symbols': ['Fe', 'Mn'], 'terms': ['mie_Fe-Mn.table', 'ENTRY']},
    {'symbols': ['Fe', 'Ni'], 'terms': ['mie_Fe-Ni.table', 'ENTRY']},
    {'symbols': ['Mn', 'Ni'], 'terms': ['mie_Mn-Ni.table', 'ENTRY']},
]

List files assigned to the potential for options #2-4

In [23]:
for file in filenames:
    print(file.name)

library.meam
Fe.meam


Option #2 For potentials with single parameter files, e.g. eam/alloy

- __paramfile__ (*str*) The name (no path) of the potential's parameter file.

In [29]:
kwargs['paramfile'] = 'GaAsAu.lammps'

Option #3 For potentials with a parameter file and a library file, e.g. meam

- __libfile__ (*str*) The name (no path) of the potential's library parameter file.
- __paramfile__ (*str, optional*) The name (no path) of the potential's parameter file, if one is used.

In [24]:
kwargs['libfile'] = 'library.meam'
kwargs['paramfile'] = 'Fe.meam'

Option #4 For classic EAM potentials, i.e. eam

- __paramfiles__ (*list*) The names (no paths) of the parameter files for each unique element. Order should match order of elements.

In [ ]:
kwargs['paramfiles'] = []

Option #5 For OpenKIM potentials

- __kimid__ (*str*) The KIM model ID.

In [ ]:
kwargs['kimid'] = 

#### Build potential_LAMMPS

In [25]:
lammps_potential = potentials.build_lammps_potential(**kwargs).potential()

In [28]:
# Custom select the "style" if needed
lammps_potential = potentials.buildrecord.potential_LAMMPS.ParamFileBuilder(**kwargs).potential()

### 5.2 Load potential_LAMMPS from database 

In [ ]:
#lammps_potential = db.get_lammps_potential(id='2023--Jana-R--Fe--LAMMPS--ipr1', verbose=True)

### 5.3 Load potential_LAMMPS from local file not in database 

In [44]:
#fname = Path('2020--Groger-R--Co-Cr-Fe-Mn-Ni--LAMMPS--ipr1.json')
#lammps_potential = potentials.record.PotentialLAMMPS(fname)

### 5.4 Review and save

#### Review model

In [26]:
print(lammps_potential.model.json(indent=2))

{
  "potential-LAMMPS": {
    "key": "901d4565-8f74-4b8d-bbc0-e6fe1ac4f76b",
    "id": "2025--Sharifi-H--Fe--LAMMPS--ipr1",
    "potential": {
      "key": "db96e622-2fd1-4a7a-a8e7-c0995caf5891",
      "id": "2025--Sharifi-H-Wick-C-D--Fe"
    },
    "comments": "Potential 2025--Sharifi-H--Fe--LAMMPS--ipr1 listed in the NIST Interatomic Potentials Repository:\nhttps://www.ctcms.nist.gov/potentials/entry/2025--Sharifi-H-Wick-C-D--Fe/2025--Sharifi-H--Fe--LAMMPS--ipr1.html\n",
    "units": "metal",
    "atom_style": "atomic",
    "atom": {
      "symbol": "Fe",
      "element": "Fe",
      "mass": 55.845
    },
    "pair_style": {
      "type": "meam"
    },
    "pair_coeff": {
      "term": [
        {
          "file": "library.meam"
        },
        {
          "option": "Fe"
        },
        {
          "file": "Fe.meam"
        },
        {
          "symbols": true
        }
      ]
    },
    "artifact": [
      {
        "web-link": {
          "URL": "https://www.ctcms.nist.go

#### Save

In [27]:
db.save_lammps_potential(lammps_potential, filenames=filenames,  verbose=True)
with db.remote_database.cdcs.auto_set_pid_off():
    db.upload_lammps_potential(lammps_potential, workspace=workspace, overwrite=True, verbose=True)

potential_LAMMPS record named 2025--Sharifi-H--Fe--LAMMPS--ipr1 added to C:\Users\lmh1\Documents\library
files saved to local folder
potential_LAMMPS record named 2025--Sharifi-H--Fe--LAMMPS--ipr1 added to https://potentials.nist.gov/
potential_LAMMPS record named 2025--Sharifi-H--Fe--LAMMPS--ipr1 assigned to workspace Global Public Workspace


In [ ]:
#db.delete_lammps_potential(lammps_potential, local=True, remote=True, verbose=True)

In [ ]:
#db.download_lammps_potentials(getfiles=True, overwrite=True, verbose=True)

- - -

## 5. Actions

Actions are used by the website to note changes in content and representation. 

### 5.1 Build action

#### Option #1: new posting(s)

In [35]:
for complete_potential in complete_potentials:
    print(complete_potential.id)

2025--Sharifi-H-Wick-C-D--Fe-Mn-Ni-Ti-Cu-Cr-Co-Al


In [36]:
action = potentials.load_record('Action',
    type = 'new posting',
    potentials = complete_potentials,
    #date = datetime.date(2021, 10, 26),
    comment = "New MEAM potential for Fe-Mn-Ni-Ti-Cu-Cr-Co-Al",
)

#### Option #2: updated posting(s)

In [39]:
action = potentials.load_record('Action',
    type = 'updated posting',
    potentials = complete_potentials,
    date = None,
    comment = 'New version added that fixes the atomic mass of H',
)

#### Option #3: retraction

In [ ]:
action = potentials.load_record('Action',
    type = 'retraction',
    potentials = complete_potentials,
    date = None,        
    comment =,
)

#### Option #4: site change

In [ ]:
action = potentials.load_record('Action',
    type = 'site change',
    date = None,
    comment = 'IDs added for all non-LAMMPS implementations. The handling of fictional potentials made consistent with "real" potentials.',
    potentials = []
)

### 5.2 Look and save

In [37]:
action.build_model()
action.html(render=True)

In [38]:
db.upload_action(action, workspace=workspace, overwrite=True, verbose=True)

Action record named 2025-02-14 New MEAM potential for Fe-Mn-Ni-Ti-Cu-Cr-Co-Al added to https://potentials.nist.gov/
Action record named 2025-02-14 New MEAM potential for Fe-Mn-Ni-Ti-Cu-Cr-Co-Al assigned to workspace Global Public Workspace


In [29]:
action = db.get_action(name='2025-02-14 New MEAM potential for Fe-Mn-Ni-Ti-Cu-Cr-Co-Al')

In [33]:
action.potentials[0].metadata()

{'id': '2025--Sharifi-H-Wick-C-D--Fe-Mn-Ni-Ti-Cu-Cr-Co-Al',
 'key': '3f1437f6-f3ad-4326-a186-505cbfef79d9',
 'dois': [],
 'elements': ['Fe', 'Mn', 'Ni', 'Ti', 'Cu', 'Cr', 'Co', 'Al'],
 'othername': None,
 'fictional': False}

- - -

## 6. Requests

Requests are used by the website to list when people are looking for specific models that we don't have.

### 6.1 Build request

In [42]:
request = potentials.load_record(
    'Request',
    #comment = 'for WS2 films',
    #date = datetime.date(2022, 11, 1),
    systems = [
        {
        #    'formula': 'KNbO3',
            'elements': ['W','Co', 'C'],
        },
        #{
        #    'formula': 'RbNbO3',
        #    'elements': ['Rb', 'Nb', 'O'],
        #},
    ]
)


### 6.2 Look and save

In [43]:
request.build_model()
display(HTML(request.html()))

In [44]:
db.upload_request(request, workspace=workspace, overwrite=True, verbose=True)

Request record named 2024-03-21 W Co C added to https://potentials.nist.gov/
Request record named 2024-03-21 W Co C assigned to workspace Global Public Workspace


## 7. Related models

In [34]:
import json

def add_related_models(localpath, interaction, model1, model2):
    
    with open(Path(localpath, 'site/related-interactions.json')) as f:
        related_models = json.load(f)
    
    # If no shared models for that interaction yet
    if interaction not in related_models:
        print('new interaction')
        related_models[interaction] = [[model1, model2]]
            
    else:
        int_models = related_models[interaction]
        
        # Search for set with one of the elements
        match = False
        for i in range(len(int_models)):
            modelset = int_models[i]
            if model1 in modelset:
                if match is False:
                    match = i
                else:
                    raise ValueError('given models found in multiple sets!')
            if model2 in modelset:
                if match is False or match == i:
                    match = i
                else:
                    raise ValueError('given models found in multiple sets!')
                    
        if match is False:
            print('new set')
            int_models.append(sorted([model1, model2]))
        else:
            if model1 not in int_models[match]:
                int_models[match] = sorted(int_models[match] + [model1])
                print('model1 added to existing set')
            elif model2 not in int_models[match]:
                int_models[match] = sorted(int_models[match] + [model2])
                print('model2 added to existing set')
            else:
                print('both models already in existing set')
    
    with open(Path(localpath, 'site/related-interactions.json'), 'w') as f:
        json.dump(related_models, fp=f)

In [44]:
interaction = 'Al-Ni-Ti'
model1 = '2024--Huang-S-Xiong-Y-Ma-S-et-al--Ni-Al-Co-Ti'
model2 = '2017--Kim-Y-K-Kim-H-K-Jung-W-S-Lee-B-J--Ni-Al-Ti'
machine = 'desktop'

if machine == 'desktop':
    localpath = Path('F:/website/IPR-website/potentials/')
elif machine == 'laptop':
    localpath = Path('C:/Users/lmh1/Documents/website/IPR-website/potentials/')

add_related_models(localpath, interaction, model1, model2)

new interaction


In [41]:
localpath = Path('F:/website/IPR-website/potentials/')

elements = sorted(['Fe', 'Mn', 'Ni', 'Ti', 'Cu', 'Cr', 'Co', 'Al'])





In [42]:
elements

['Al', 'Co', 'Cr', 'Cu', 'Fe', 'Mn', 'Ni', 'Ti']

In [36]:
r, df = db.get_potentials(author='Sharifi', return_df=True)

In [40]:
potids = df.id.tolist()

In [48]:
for element in elements:
    search = f'-{element}'
    first = None
    count = 0
    for potid in potids:
        if search in potid:
            count += 1
            if first is None:
                first = potid
            else:
                #print(element, first, potid)
                add_related_models(localpath, element, first, potid)
    print(count)

new set
model2 added to existing set
model2 added to existing set
model2 added to existing set
model2 added to existing set
model2 added to existing set
model2 added to existing set
model2 added to existing set
model2 added to existing set
model2 added to existing set
model2 added to existing set
model2 added to existing set
model2 added to existing set
model2 added to existing set
model2 added to existing set
model2 added to existing set
model2 added to existing set
model2 added to existing set
model2 added to existing set
model2 added to existing set
model2 added to existing set
model2 added to existing set
model2 added to existing set
model2 added to existing set
model2 added to existing set
model2 added to existing set
model2 added to existing set
model2 added to existing set
model2 added to existing set
30
new set
model2 added to existing set
model2 added to existing set
model2 added to existing set
model2 added to existing set
model2 added to existing set
model2 added to existing

In [54]:
for i in range(len(elements)):
    for j in range(i+1, len(elements)):
        for k in range(j+1, len(elements)):
            element1 = elements[i]
            element2 = elements[j]
            element3 = elements[k]
            interaction = f'{element1}-{element2}-{element3}'
            search1 = f'-{element1}'
            search2 = f'-{element2}'
            search3 = f'-{element3}'
            first = None
            count = 0
            for potid in potids:
                if search1 in potid and search2 in potid and search3 in potid:
                    count += 1
                    if first is None:
                        first = potid
                    else:
                        #print(interaction, first, potid)
                        add_related_models(localpath, interaction, first, potid)
            print(count)

new interaction
2
new interaction
2
new interaction
2
new interaction
2
new set
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new set
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
new interaction
2
